# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

10# Load the dataset [10 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
train_bodies = pd.read_csv(project_path+'train_bodies.csv')
train_stances = pd.read_csv(project_path+'train_stances.csv')

In [0]:
dataset = pd.merge(train_bodies, train_stances, on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [8]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [10 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [11]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = Tokenizer(MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
texts = []
for text in dataset['articleBody']:
  texts.append(text)
for text in dataset['Headline']:
  texts.append(text)
t.fit_on_texts(texts)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
from nltk.tokenize import sent_tokenize
texts = []
articles = []

In [0]:
for text in dataset['articleBody']:
  texts.append(text)
  sentences = sent_tokenize(text)
  articles.append(sentences)

## Check 2:

first element of texts and articles should be as given below. 

In [16]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [17]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [10 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence
data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH))

for i, sentences in enumerate(articles):
  for j, sent in enumerate(sentences):
    if j < MAX_SENTS:
      wordTokens = text_to_word_sequence(sent)
      k = 0
      for _, word in enumerate(wordTokens):
        if k < MAX_SENT_LENGTH and word in t.word_index and t.word_index[word] < MAX_NB_WORDS:
          data[i, j, k] = t.word_index[word]
          k = k+1

### Check 3:

Accessing first element in data should give something like given below.

In [19]:
data[0, :, :]

array([[3.0000e+00, 4.8100e+02, 4.2700e+02, 7.2110e+03, 8.1000e+01,
        3.0000e+00, 3.7330e+03, 3.3100e+02, 5.0000e+00, 3.8910e+03,
        3.5000e+02, 4.0000e+00, 1.4310e+03, 2.9580e+03, 1.0000e+00,
        8.9000e+01, 1.2000e+01, 4.6400e+02, 0.0000e+00, 0.0000e+00],
       [7.5800e+02, 9.5000e+01, 1.0470e+03, 3.0000e+00, 2.6790e+03,
        1.7520e+03, 7.0000e+00, 1.8900e+02, 3.0000e+00, 1.2170e+03,
        1.0750e+03, 2.0300e+03, 7.0000e+02, 1.5900e+02, 1.0000e+00,
        3.0320e+03, 4.4800e+02, 1.0000e+00, 5.5500e+02, 2.3500e+02],
       [8.9000e+01, 1.0670e+03, 4.1150e+03, 2.3490e+03, 1.2000e+01,
        3.0000e+00, 1.0920e+03, 3.3060e+03, 1.9000e+01, 1.0000e+00,
        8.9000e+01, 2.0000e+00, 1.7930e+03, 1.0000e+00, 5.2100e+02,
        2.0090e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.1110e+03],
       [1.8100e+02, 3.6400e+03, 9.7200e+02, 2.0000e+02, 2.5560e+03,
        4.4000e+01, 6.7750e+03, 1.7220e+03, 1.2520e+03, 5.0000e+00,
        1.3317e+04, 1.7936e+04, 1.0000e+00, 7

10# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [10 marks] 

In [0]:
texts_heading = []
articles_heading = []

for text in dataset['Headline']:
  texts_heading.append(text)
  sentences = sent_tokenize(text)
  articles_heading.append(sentences)

data_heading = np.zeros((len(articles_heading), MAX_SENTS_HEADING, MAX_SENT_LENGTH))

for i, sentences in enumerate(articles_heading):
  for j, sent in enumerate(sentences):
    if j < MAX_SENTS_HEADING:
      wordTokens = text_to_word_sequence(sent)
      k = 0
      for _, word in enumerate(wordTokens):
        if k < MAX_SENT_LENGTH and word in t.word_index and t.word_index[word] < MAX_NB_WORDS:
          data_heading[i, j, k] = t.word_index[word]
          k = k+1

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = np.array(pd.get_dummies(dataset['Stance']))

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [22]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

x_train, x_val, x_heading_train, x_heading_val, y_train, y_val = train_test_split(data, data_heading, labels, test_size=0.2, random_state=42)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [26]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [27]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(t.word_index.items()), 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [20 marks]  

### Import layers from Keras to build the model

In [0]:
from tensorflow.keras.layers import Input, Embedding, Dropout, LSTM, Dot, Dense, Reshape, BatchNormalization
from tensorflow.keras.models import Model

### Model

In [0]:
#Encoder part 
inputs1 = Input(shape=(x_train.shape[1], x_train.shape[2], ))
fe1 = Embedding(len(t.word_index.items()), 100, mask_zero=True)(inputs1)
fe2 = Dropout(0.5)(fe1)
fe3 = Reshape((400, 100))(fe2)
fe4 = LSTM(128)(fe3)

#Decoder part
#partial caption sequence model
inputs2 = Input(shape=(x_heading_train.shape[1], x_heading_train.shape[2],))
se1 = Embedding(len(t.word_index.items()), 100, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = Reshape((20, 100))(se2)
se4 = LSTM(128)(se3)

# Decoder feedforward 
decoder1 = Dot(1, normalize=True)([fe4, se4])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(4, activation='softmax')(decoder2)

#merging two models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1, 20)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 20, 100)  2787300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 20, 100)   2787300     input_2[0][0]                    
______________________________________________________________________________________________

In [0]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.layers[3].set_weights([embedding_matrix])
model.layers[3].trainable = False

In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1, 20)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 20, 100)  2787300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 20, 100)   2787300     input_2[0][0]                    
______________________________________________________________________________________________

### Compile and fit the model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Saving the model after every Epoch using ModelCheckpoint callback
model_checkpoint=ModelCheckpoint('FNC.h5', 
                                 save_best_only=False,
                                 verbose=1)

In [35]:
model.fit([x_train, x_heading_train], y_train, epochs = 50, validation_data=([x_val, x_heading_val], y_val), callbacks=[model_checkpoint])

Epoch 1/50
1248/1250 [============================>.] - ETA: 0s - loss: 0.8095
Epoch 00001: saving model to FNC.h5
1250/1250 [==============================] - 34s 27ms/step - loss: 0.8092 - val_loss: 0.7912
Epoch 2/50
1250/1250 [==============================] - ETA: 0s - loss: 0.7924
Epoch 00002: saving model to FNC.h5
1250/1250 [==============================] - 33s 27ms/step - loss: 0.7924 - val_loss: 0.7759
Epoch 3/50
1249/1250 [============================>.] - ETA: 0s - loss: 0.7791
Epoch 00003: saving model to FNC.h5
1250/1250 [==============================] - 33s 26ms/step - loss: 0.7794 - val_loss: 0.7607
Epoch 4/50
1249/1250 [============================>.] - ETA: 0s - loss: 0.7658
Epoch 00004: saving model to FNC.h5
1250/1250 [==============================] - 33s 26ms/step - loss: 0.7658 - val_loss: 0.7474
Epoch 5/50
1250/1250 [==============================] - ETA: 0s - loss: 0.7251
Epoch 00005: saving model to FNC.h5
1250/1250 [==============================] - 33s 26ms

In [0]:
y_pred = model.predict([x_val, x_heading_val])

In [0]:
y_pred = np.argmax(y_pred, axis=1)

In [0]:
y_val = np.argmax(y_val, axis=1)

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, y_pred)

array([[ 497,    0,  170,   49],
       [ 111,    0,   38,    7],
       [ 162,    0, 1613,   34],
       [ 158,    0,   76, 7080]])

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.9194597298649325